#### Loading data sources

In [189]:
!pip install imblearn
!pip install imbalanced-learn
!pip install patsy

Looking in indexes: https://pypi.python.org/simple, https://cognite.jfrog.io/cognite/api/pypi/snakepit/simple
Looking in indexes: https://pypi.python.org/simple, https://cognite.jfrog.io/cognite/api/pypi/snakepit/simple
Looking in indexes: https://pypi.python.org/simple, https://cognite.jfrog.io/cognite/api/pypi/snakepit/simple


In [190]:
# This Jupyter Notebook does feature engineering joining all features in all tables.

import matplotlib.pyplot as plt
import pandas as pd
import seaborn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score
from datetime import datetime
from imblearn.over_sampling import SMOTE
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,roc_curve,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)


#### Parameters

#### Load the dataset

In [191]:
# Read employees dataset
df_customer_raw = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [192]:
df_customer_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [193]:
df_customer = df_customer_raw

In [194]:
df_customer.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [195]:
label_encoded_fields = [ 'Partner', 'Dependents', 'PhoneService', 'Churn', 'PaperlessBilling' ]
one_hot_encoded_fields = [ 'gender', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingMovies', 'StreamingTV' ]

In [196]:
print('Before Label Encoding')

for field in label_encoded_fields:
    print(field)
    print(df_customer[field].unique())

Before Label Encoding
SeniorCitizen
[0 1]
Partner
['Yes' 'No']
Dependents
['No' 'Yes']
PhoneService
['No' 'Yes']
Churn
['No' 'Yes']
PaperlessBilling
['Yes' 'No']


In [197]:
print('After Label Encoding')

for field in label_encoded_fields:
    df_customer[field] = le.fit_transform(df_customer[field])
    print(f'Field: {field}')
    print(df_customer[field].unique())

After Label Encoding
Field: SeniorCitizen
[0 1]
Field: Partner
[1 0]
Field: Dependents
[0 1]
Field: PhoneService
[0 1]
Field: Churn
[0 1]
Field: PaperlessBilling
[1 0]


In [198]:
print('Before One Hot Encoding')

for field in one_hot_encoded_fields:
    print(field)
    print(df_customer[field].unique())

Before One Hot Encoding
gender
['Female' 'Male']
OnlineSecurity
['No' 'Yes' 'No internet service']
OnlineBackup
['Yes' 'No' 'No internet service']
DeviceProtection
['No' 'Yes' 'No internet service']
TechSupport
['No' 'Yes' 'No internet service']
StreamingMovies
['No' 'Yes' 'No internet service']
StreamingTV
['No' 'Yes' 'No internet service']


In [199]:
print('After One Hot Encoding')

for field in one_hot_encoded_fields:
    encoded_data = ohe.fit_transform(df_customer[[field]])
    encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out([field]))
    df_customer = pd.concat([df_customer, encoded_df], axis=1)
    df_customer.drop(columns=[field], inplace=True)


df_customer.info()

After One Hot Encoding
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   customerID                            7043 non-null   object 
 1   SeniorCitizen                         7043 non-null   int64  
 2   Partner                               7043 non-null   int64  
 3   Dependents                            7043 non-null   int64  
 4   tenure                                7043 non-null   int64  
 5   PhoneService                          7043 non-null   int64  
 6   MultipleLines                         7043 non-null   object 
 7   InternetService                       7043 non-null   object 
 8   Contract                              7043 non-null   object 
 9   PaperlessBilling                      7043 non-null   int64  
 10  PaymentMethod                         7043 non-null   object 

df_customer['StreamingTV'].unique()

In [200]:
df_customer

,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,Contract,PaperlessBilling,...,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes
0,7590-VHVEG,0,1,0,1,0,No phone service,DSL,Month-to-month,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,5575-GNVDE,0,0,0,34,1,No,DSL,One year,0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3668-QPYBK,0,0,0,2,1,No,DSL,Month-to-month,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,7795-CFOCW,0,0,0,45,0,No phone service,DSL,One year,0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,9237-HQITU,0,0,0,2,1,No,Fiber optic,Month-to-month,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0,1,1,24,1,Yes,DSL,One year,1,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
7039,2234-XADUH,0,1,1,72,1,Yes,Fiber optic,One year,1,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7040,4801-JZAZL,0,1,1,11,0,No phone service,DSL,Month-to-month,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7041,8361-LTMKD,1,1,0,4,1,Yes,Fiber optic,Month-to-month,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [201]:
df_customer = df_customer[['Partner', 'Dependents', 'PhoneService', 'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes', 'Churn']]

In [202]:
X = df_customer.drop(columns=['Churn'])  # Features
y = df_customer['Churn']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)

y_predicted = gb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_predicted)
report = classification_report(y_test, y_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1036
           1       0.00      0.00      0.00       373

    accuracy                           0.74      1409
   macro avg       0.37      0.50      0.42      1409
weighted avg       0.54      0.74      0.62      1409



/home/priyanka/etc/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyanka/etc/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/priyanka/etc/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
